# Section 4: Run quantum circuits

## TASK 4.1: Demonstrate an understanding of execution modes such as: session with dedicated, priority, and batch mode

1. Which statement best distinguishes the three Qiskit Runtime execution modes?

a. Job runs a single primitive request; Batch runs many independent jobs submitted all at once; Session provides a time window with exclusive access to a specific QPU.<br>
b. Job and Batch are identical, but Session only works on simulators.<br>
c. Session guarantees lower queue time for the very first job; Batch guarantees exclusive access.<br>
d. Batch is only for variational algorithms; Session is only for tomography.<br>

:::{dropdown}answer 
The answer is a. 

Job mode is for one-off primitive calls, Batch is optimized for multi-job workloads with all inputs ready, and Session opens a dedicated window with exclusive access to the QPU for iterative workloads.

| Feature                                                        | Job                                                                                                             | Batch                                                                                                                                              | Session                                                                                                                                                                                              |
| -------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **When first job enters queue**                                | Just like a normal job; you incur full queue delay for that job.                                                | Same for the *first* job of the batch.                                                                                         | Same: queue for the first job.                                                                                                                                                   |
| **Overhead per job**                                           | Each job has its own setup, queueing etc.                                                                       | Reduced overhead for multiple jobs — classical preprocessing parallelized; jobs are grouped.                                   | Reduced overhead between jobs; fewer gaps; more predictable performance.                                                                                                                             |
| **QPU utilization / idle time**                                | You may get gaps if multiple jobs are submitted separately; QPU may be idle between them.                       | Aim is to minimize idle time between your jobs in the batch.                                                                   | Similarly, you reserve the QPU for your session, so minimal idle or interference.                                                                                                                    |
| **Exclusivity / interference from other users / calibrations** | None — everything is regular shared queue.                                                                      | Not exclusive; other users’ jobs may run in parallel if capacity exists; calibration jobs might interrupt between batch jobs.  | Exclusive: no other users’ jobs, no calibration jobs in between.                                                                                                                 |
| **Suitability**                                                | Good for single circuits or simple experiments. When you don't have many jobs or don’t need strict performance. | Good when you have many independent jobs and want to reduce per-job overhead.                                                                      | Good for iterative or interactive workloads, or when you need low latency / consistency / no surprises between jobs.                                                                                 |
| **Constraints / limits**                                       | Very flexible; minimal constraints.                                                                             | Batches still share the normal queue at the start; you haven’t exclusive access; also, jobs may not run in the order submitted. | You get a session window with a *maximum time to live* (TTL). If that is exceeded, remaining jobs may fail; also some user classes (Open Plan users) may not be allowed sessions. |

ref: [Introduction to execution modes | IBM Quantum Documentation](https://quantum.cloud.ibm.com/docs/en/guides/execution-modes)

:::

2. In Session mode, what does "dedicated" access mean?

a. Your jobs are scheduled before every other user, but calibration jobs may still interrupt execution.<br>
b. Your workload has exclusive access to the selected backend during the active window; no other users' jobs or calibrations run during that time.<br>
c. Your jobs run only on a simulator with higher concurrency.<br>
d. Your batch is guaranteed to start immediately after submission.<br>

:::{dropdown}answer 
The answer is b. 

A session grants an exclusive, dedicated execution window on the chosen QPU. During this window, no other users' jobs (including calibration jobs) are interleaved. 
:::

3. Which statement correctly describes "priority" in the context of sessions?

a. Closing a session cancels all queued jobs immediately without exception.<br>
b. When a session is closed (not canceled), it stops accepting new jobs but continues to execute already queued session jobs with priority until completion.<br>
c. Priority means sessions preempt running jobs from other users mid-shot.<br>
d. Priority is a separate execution mode unrelated to sessions.<br>

:::{dropdown}answer 
The answer is b. 

If you close (not cancel) a session, it no longer accepts new jobs, but the jobs already queued in that session continue and are treated with priority until they finish. 
:::

4. Which is true about Batch mode?

a. It gives exclusive access to the QPU for the batch window.<br>
b. It shortens first-job queuing time by skipping the normal queue.<br>
c. It can parallelize or thread classical pre-processing across jobs, then pack quantum execution tightly; however, batches do *not* get exclusive access.<br>
d. It requires iterative feedback between jobs and therefore cannot be used for independent jobs.<br>

:::{dropdown}answer 
The answer is c. 

Batch mode improves throughput by parallelizing classical pre-processing and tightly sequencing QPU execution across multiple independent jobs, but it does not provide exclusive access to the QPU. 
:::

5. How is usage (cost) accounted for across modes?

a. Session usage equals only the sum of quantum time; Batch and Job usage count wall-clock time.<br>
b. Session usage is wall-clock time from first job start until session termination; Batch usage is the sum of quantum time across batch jobs; Job usage is quantum time of that job.<br>
c. All modes charge only for the number of shots requested.<br>
d. Batch usage is wall-clock time; Session usage is quantum time only.<br>

:::{dropdown}answer 
The answer is b. 

Session usage is the full wall-clock window from first job start until the session ends; Batch usage sums quantum time of the batch's jobs; a single Job is charged by its quantum time. 
:::

6. Which statement about plan restrictions and best practices is correct?

a. Open Plan users can freely run session jobs; Premium users cannot.<br>
b. Open Plan users cannot submit session jobs; generally, prefer Batch for multi-job workloads unless you need the added benefits of a Session.<br>
c. Batch mode is unavailable on paid plans.<br>
d. Job mode is deprecated and should be avoided.<br>

:::{dropdown}answer 
The answer is b. 

Open Plan users are restricted from using sessions. For most multi-job workloads with inputs ready, Batch is recommended; use Session when you need the dedicated window or iterative interaction. 
:::

7. What happens if no jobs are ready within the "interactive TTL" during a batch or session?

a. The workload is temporarily deactivated; later jobs can reactivate it as long as maximum TTL has not been reached.<br>
b. The workload is immediately deleted and cannot be resumed.<br>
c. All queued jobs are canceled automatically.<br>
d. Interactive TTL only applies to Job mode, not Batch or Session.<br>

:::{dropdown}answer 
The answer is a. 

If the interactive TTL elapses with no jobs ready, the batch or session becomes inactive. A later job can reactivate it through the normal queue, provided the maximum TTL has not expired. 
:::

8. Which statement about parallelism and ordering is accurate?

a. In Batch mode, jobs always run strictly in the submission order.<br>
b. In Session mode, you cannot run jobs in parallel.<br>
c. In both Batch and Session modes, multiple jobs can run in parallel subject to backend parallelism; batches do not guarantee submission order.<br>
d. Neither Batch nor Session supports parallel execution across jobs.<br>

:::{dropdown}answer 
The answer is c. 

Backends can execute multiple jobs in parallel. Batches do not guarantee that jobs run in submission order; sessions can also exploit available parallelism. 
:::

9. When creating a session through the REST API, which `mode` values are supported and what do they imply?

a. `mode` can be `dedicated` for exclusive access or `batch` for a session scheduled like a batch (non-exclusive), depending on needs and plan.<br>
b. `mode` must always be `session`; it has no other valid values.<br>
c. `mode` accepts `priority` and `economy` only.<br>
d. `mode` is optional and ignored by the platform.<br>

:::{dropdown}answer 
The answer is a. 

The sessions API accepts a `mode` field with values such as `dedicated` (exclusive access) or `batch`. These control how the session is scheduled on the backend. 
:::

10. Which statement about the first job in Batch or Session modes is correct?

a. It skips the normal queue to start immediately.<br>
b. It enters the normal queue; these modes do not reduce the first job's queuing time.<br>
c. It is always preemptive over calibration jobs.<br>
d. It is scheduled only after all other users' jobs complete.<br>

:::{dropdown}answer 
The answer is b. 

For both Batch and Session, the first job goes through the normal queue. These modes help after the first job starts (packing jobs or providing a dedicated window), not in bypassing the initial queue. 
:::

11. You have 60 independent primitive jobs (no data dependency between them), and all inputs are ready. Which execution mode best fits this workload and why?

a. Session mode, because it provides an exclusive window and reduces first‑job queueing time.<br>
b. Batch mode, because it lets you submit many independent jobs at once and exploit parallel processing without requiring exclusivity.<br>
c. Job mode, because it guarantees strict FIFO across all users.<br>
d. Session mode, because interactive TTL is configurable per user to hours.<br>

:::{dropdown}answer 
The answer is b. 

Batch is intended for non‑iterative multi‑job workloads whose jobs can run independently. It enables parallel processing and tight packing on the backend without requiring dedicated, exclusive access. 
:::

12. In both Batch and Session modes, how is the very first job scheduled?

a. It bypasses the normal queue and starts immediately.<br>
b. It enters the normal queue like any other job; modes primarily help after the first job begins.<br>
c. It preempts calibration jobs on the backend.<br>
d. It is delayed until the batch/session accumulates at least five jobs.<br>

:::{dropdown}answer 
The answer is b. 

Neither Batch nor Session short‑circuits the platform queue for the first job. Their benefits appear after the first job starts (parallel packing for Batch; dedicated window for Session). 
:::

13. Which statement about Batch TTLs is correct?

a. Interactive TTL in Batch is user‑configurable per instance.<br>
b. When max TTL is reached, running jobs finish, but queued jobs are failed; interactive TTL being idle temporarily deactivates the batch.<br>
c. When max TTL is reached, all jobs (running and queued) are canceled immediately.<br>
d. Interactive TTL applies only to sessions, not batches.<br>

:::{dropdown}answer 
The answer is b. 

You can set a batch's `max_time` (max TTL). When reached, the batch closes: running jobs complete, queued jobs fail. There is also a fixed interactive TTL; if no jobs are queued within that window, the batch becomes temporarily inactive. 
:::

14. You are tuning parameters in a loop, where each iteration submits a small job that depends on the previous results. Which mode should you prefer?

a. Batch mode, because it is designed for independent jobs with no conditional relation.<br>
b. Session mode, because it is designed for iterative workloads and groups multiple jobs into one dedicated window.<br>
c. Job mode, because it gives the finest control over each iteration's queueing.<br>
d. Either Session or Batch; they are equivalent for iterative feedback.<br>

:::{dropdown}answer 
The answer is b. 

Session mode is for iterative workloads where later jobs depend on earlier results, allowing efficient back‑and‑forth within a dedicated window. 
:::

15. Which is true about ordering and parallelism when using Batch mode?

a. Jobs always execute strictly in submission order.<br>
b. Jobs may execute in parallel subject to backend capacity, and order is not guaranteed.<br>
c. Jobs cannot run in parallel in Batch mode.<br>
d. Order is guaranteed only if all jobs target the same primitive.<br>

:::{dropdown}answer 
The answer is b. 

Batch leverages backend parallelism when available. It does not guarantee submission order; the scheduler can run multiple jobs concurrently. 
:::

16. What happens when a Batch context manager (`with Batch(...)`) exits or when `batch.close()` is called?

a. The batch cancels all pending jobs immediately.<br>
b. The batch stops accepting new jobs but lets running and already‑queued jobs complete (until TTL), then closes.<br>
c. The batch remains open until manually canceled via the REST API.<br>
d. The batch converts remaining jobs to a Session automatically.<br>

:::{dropdown}answer 
The answer is b. 

Leaving the context or calling `close()` stops new job submissions. Running and queued jobs continue to completion (subject to TTL), after which the batch is closed. 
:::

17. Which statement about usage accounting best reflects Batch versus Session?

a. Batch usage is the sum of quantum time across batch jobs; Session usage is the wall‑clock time of the session window from its first job until it ends or goes inactive.<br>
b. Both Batch and Session usage are calculated purely by shot count.<br>
c. Batch usage is wall‑clock time; Session usage is only quantum time.<br>
d. Both modes charge only while a job is running on the QPU, excluding classical time.<br>

:::{dropdown}answer 
The answer is a. 

In Batch, usage is the sum of quantum time used by jobs in the batch. In Session, usage is the wall‑clock window starting from the first job until it ends/goes inactive (includes classical and quantum). 
:::

18. Inside a Batch or Session context, which configuration causes your primitive call to run in plain Job mode instead?

a. Passing `mode=batch` (or `mode=session`) when creating the primitive.<br>
b. Setting `backend=backend` directly on the primitive call, which overrides the context.<br>
c. Calling `batch.details()` before the first submission.<br>
d. Using different primitives (Sampler vs Estimator) in one batch.<br>

:::{dropdown}answer 
The answer is b. 

Specifying `backend=backend` on the primitive forces job mode even inside a Batch or Session context. Prefer the `mode=` parameter to bind the primitive to the context. 
:::

19. Which guidance aligns with choosing the execution mode?

a. Use Batch when you can prepare many independent jobs and want higher throughput; use Session when you need iterative feedback or a dedicated window.<br>
b. Prefer Session for all workloads because it guarantees earliest start time.<br>
c. Prefer Job mode for multi‑job workloads to maximize parallelism.<br>
d. Use Batch only when jobs share the same transpilation options.<br>

:::{dropdown}answer 
The answer is a. 

Batch targets non‑iterative multi‑job workloads; Session targets iterative workloads and provides a dedicated execution window. Job mode is for one‑off jobs. 
:::

20. How can you inspect a batch’s configuration (including TTL values) and current status from code?

a. Call `batch.details()` to retrieve configuration and status, including interactive and max TTL values.<br>
b. Call `service.batch_status()` which returns TTL values for every batch in the instance.<br>
c. Use `sampler.options()` to see the batch’s TTLs.<br>
d. It’s not possible to retrieve TTL values programmatically.<br>

:::{dropdown}answer 
The answer is a. 

`batch.details()` provides a comprehensive overview of the batch, including interactive and max TTL values and current status. 
:::

---

## TASK 4.2: Demonstrate understanding of how to run quantum circuits with real hardware using Qiskit Runtime primitives and applying broadcasting rules

1. Which tuple structure is a valid Primitive Unified Bloc (PUB) for the **SamplerV2** when running on real hardware?

a. A single `QuantumCircuit`, a parameter value collection (if needed), and an optional shot count<br>
b. A list of `QuantumCircuit` objects, an observable, and a target precision<br>
c. A single `QuantumCircuit` and a list of `SparsePauliOp` observables<br>
d. Only a list of parameter values and a shot count<br>


:::{dropdown}answer 
The answer is a. 

The Sampler PUB contains at most three values: a single circuit (with measurements), an optional collection of parameter values to bind `Parameter`s at runtime, and an optional number of shots. 
:::


2. In the **EstimatorV2** PUB, which elements are required to estimate expectation values on IBM Quantum hardware?

a. A circuit, one or more observables (as an array-like), and optionally parameter values and a target precision<br>
b. Only a circuit with measurements and a shot count<br>
c. A list of circuits and a single observable string like `'ZZ'`<br>
d. Just parameter values and a backend name<br>


:::{dropdown}answer 
The answer is a. 

Estimator PUBs include a circuit, a list/array of observables, optional parameter bindings, and optional target precision. 
:::


3. A circuit has two `Parameter`s. You pass parameter bindings shaped `(100, 2)` and an observables array shaped `(3, 1)` to **EstimatorV2** in one PUB. What is the shape of the resulting expectation value array for that PUB?

a. (3, 100)<br>
b. (100, 3)<br>
c. (3,)<br>
d. (100,)<br>


:::{dropdown}answer 
The answer is a. 

NumPy-style broadcasting combines `(100, 2)` with `(3, 1)` to yield results shaped `(3, 100)`. 

## What is broadcasting in Qiskit

* In Qiskit’s newer *primitive unified block* (PUB) interface, when you run primitives, you often pass in inputs that include:

  1. one or more circuits
  2. one or more observables (for Estimator)
  3. sets of parameter values (“parameter bindings”)

* Broadcasting is the process by which Qiskit “aligns” or “expands” those observable arrays and parameter–value arrays so that they match in shape and can be combined pairwise to produce results. It’s analogous to **NumPy broadcasting**: you may have, say, 3 observables but 100 parameter settings, or vice versa, and broadcasting lets you compute all combinations (or appropriate combinations) without manually duplicating data.

* The result is that the output has a shape corresponding to the broadcasted shape of those input arrays. Each element in that output corresponds to a particular observables–params pair as determined by the broadcast.

---

## Broadcasting rules in Qiskit

Qiskit follows the **NumPy broadcasting rules** for combining observables and parameter-value arrays. Key rules:

1. **Different dimensions allowed**
   Input arrays (observables, parameter values) do *not* need to have the same number of dimensions. The result will have as many dimensions as the input with the largest number of dims. 

2. **Size of each dimension = maximum among inputs**
   For each dimension (counting from the rightmost / trailing dimension backwards), the output’s size is the maximum size of that dimension among the inputs. Missing dimensions are treated as having size 1. 

3. **Compatibility: either equal size or size 1**
   For each corresponding dimension (again, aligning via the trailing dims), the sizes must either match, or one of them must be 1. If neither condition holds, the arrays *cannot* broadcast.

4. **Shape starts from the rightmost dimension**
   The broadcasting comparison begins at the last dimension of each array, then moves leftwards.

## Ref
[Primitive inputs and outputs | IBM Quantum Documentation](https://quantum.cloud.ibm.com/docs/guides/primitive-input-output)

:::


4. To ensure your job runs on **real hardware** instead of a simulator using Qiskit Runtime primitives, which initialization is correct?

a. `backend = QiskitRuntimeService().least_busy(operational=True, simulator=False)` then `Estimator(mode=backend)`<br>
b. `backend = Aer.get_backend('qasm_simulator')` then `Sampler(mode=backend)`<br>
c. `Estimator(mode='ibm_oslo')` without a `QiskitRuntimeService`<br>
d. `Sampler()` with no mode or backend specified<br>


:::{dropdown}answer 
The answer is a. 

Selecting a non-simulator backend (e.g., via `least_busy(..., simulator=False)`) and passing it to `mode=` runs on real hardware. 
:::


5. Which statement about running **SamplerV2** on hardware is true?

a. Your circuit must include measurements (e.g., via `measure_all()`), otherwise no sampled bitstrings are returned<br>
b. Measurements are optional because Sampler automatically infers them<br>
c. Sampler returns expectation values and standard deviations directly<br>
d. Shots can only be set globally on the primitive, not per PUB<br>


:::{dropdown}answer 
The answer is a. 

The Sampler expects circuits with measurement instructions to produce sampled results. 
:::


6. What does **EstimatorV2** return inside each `PubResult.data` for a single PUB?

a. An object with `evs` (expectation values) and `stds` (standard deviations) matching the broadcasted shape<br>
b. A `BitArray` of shot-by-shot bytes for each classical register<br>
c. Only a scalar expectation value, without uncertainty<br>
d. A dictionary of counts keyed by bitstring<br>


:::{dropdown}answer 
The answer is a. 

Estimator PubResult `data` includes `evs` and `stds` arrays shaped according to your broadcasted inputs. 

```
└── PrimitiveResult
    ├── PubResult[0]
    │   ├── metadata
    │   └── data  ## In the form of a DataBin object
    │       ├── evs
    │       │   └── List of estimated expectation values in the shape
    |       |         specified by the first pub
    │       └── stds
    │           └── List of calculated standard deviations in the
    |                 same shape as above
    ├── PubResult[1]
    |   ├── metadata
    |   └── data  ## In the form of a DataBin object
    |       ├── evs
    |       │   └── List of estimated expectation values in the shape
    |       |        specified by the second pub
    |       └── stds
    |           └── List of calculated standard deviations in the
    |                same shape as above
    ├── ...
    ├── ...
    └── ...

```
:::


7. What does **SamplerV2** return inside each `PubResult.data` for a single PUB?

a. One `BitArray` per classical register; counts can be derived (e.g., via `get_count`)<br>
b. A single array of expectation values<br>
c. Only a histogram dictionary of counts<br>
d. A list of `SparsePauliOp` objects<br>


:::{dropdown}answer 
The answer is a. 

Sampler PubResult `data` is dict-like with a `BitArray` per `ClassicalRegister`; you can derive counts from it.

```
└── PrimitiveResult
    ├── PubResult[0]
    │   ├── metadata
    │   └── data  ## In the form of a DataBin object
    │       ├── NAME_OF_CLASSICAL_REGISTER
    │       │   └── BitArray of count data (default is 'meas')
    |       |
    │       └── NAME_OF_ANOTHER_CLASSICAL_REGISTER
    │           └── BitArray of count data (exists only if more than one
    |                 ClassicalRegister was specified in the circuit)
    ├── PubResult[1]
    |   ├── metadata
    |   └── data  ## In the form of a DataBin object
    |       └── NAME_OF_CLASSICAL_REGISTER
    |           └── BitArray of count data for second pub
    ├── ...
    ├── ...
    └── ...

```
:::


8. With broadcasting in **EstimatorV2**, how does a `SparsePauliOp` contribute to shape?

a. Each `SparsePauliOp` counts as a single scalar-like element (shape `()`), regardless of how many Pauli terms it contains<br>
b. Its shape equals the number of Pauli terms it holds<br>
c. Its shape equals the number of qubits it acts on<br>
d. It cannot be broadcast; all observables must be length-1 lists<br>


:::{dropdown}answer 
The answer is a. 

For broadcasting, each `SparsePauliOp` is treated as a single element with shape `()`. 
:::


9. Where can you specify **shots** when using **SamplerV2** with Qiskit Runtime?

a. As the optional third element in the Sampler PUB tuple<br>
b. It must be specified in the primitive constructor; PUB-level shots are not supported<br>
c. Only via environment variables on the IBM Quantum service instance<br>
d. Shots are unsupported in Runtime Sampler<br>


:::{dropdown}answer 
The answer is a. 

Sampler PUB accepts an optional shot count as its third tuple element. 
:::


10. After a job finishes, where do you find **runtime options and execution spans** for your Qiskit Runtime job?

a. In the `metadata` of the `PrimitiveResult` and each `PubResult` (e.g., execution spans for Sampler)<br>
b. Only in the `data` attribute of `PubResult`<br>
c. Only in the printed logs of the backend<br>
d. They are not exposed to the user<br>


:::{dropdown}answer 
The answer is a. 

Metadata is available at both the `PrimitiveResult` and `PubResult` levels, including execution spans for Sampler. 
:::


11. Choose the correct way to open a Qiskit Runtime session, get a backend, and create Estimator/Sampler so that the session is automatically opened and closed.

a. 
```python
from qiskit_ibm_runtime import QiskitRuntimeService, Session, SamplerV2 as Sampler, EstimatorV2 as Estimator
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
with Session(backend=backend):
    estimator = Estimator()
    sampler = Sampler()
```
b. 
```python
from qiskit_ibm_runtime import QiskitRuntimeService, Session, SamplerV2 as Sampler, EstimatorV2 as Estimator
service = QiskitRuntimeService()
backend = service.least_busy(operational=False, simulator=True)
with Session():
    estimator = Estimator(mode=backend)
    sampler = Sampler(mode=backend)
```
c. 
```python
from qiskit_ibm_runtime import QiskitRuntimeService, Session, SamplerV2 as Sampler, EstimatorV2 as Estimator
service = QiskitRuntimeService()
backend = service.least_busy()
with Session():
    estimator = Estimator(session=backend)
    sampler = Sampler(session=backend)
```
d. 
```python
from qiskit_ibm_runtime import QiskitRuntimeService, Session, SamplerV2 as Sampler, EstimatorV2 as Estimator
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
estimator = Estimator(mode=Session(backend=backend))
sampler = Sampler(mode=Session(backend=backend))
# session closes automatically when primitives are garbage-collected
```


:::{dropdown}answer 
The answer is a. 

Using `with Session(backend=...)` opens and closes the session automatically, and primitives can be instantiated with no `mode=` argument inside the context manager. The backend should be a real, operational QPU obtained via `service.least_busy(operational=True, simulator=False)`. 
This usage matches the official example for the context manager.
:::


12. Which snippet correctly sets a session time-to-live (TTL) of 25 minutes?

a.
```python
with Session(backend=backend, max_time="25m"):
    ...
```
b.
```python
with Session(backend=backend, ttl=25):
    ...
```
c.
```python
with Session(backend=backend, max_time=25):
    ...
```
d.
```python
with Session(max_time="25m"):
    ...
```


:::{dropdown}answer 
The answer is a. 

`max_time` accepts human-readable strings like `"25m"` and must be set when constructing the session with the backend. This is shown under *Session length*.
:::


13. You want to **transpile and optimize** an ansatz for a given backend before running it in a session. Which option is correct?

a.
```python
from qiskit.transpiler import generate_preset_pass_manager
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
ansatz_opt = pm.run(ansatz)
with Session(backend=backend):
    est = Estimator()
    value = est.run(ansatz_opt, ham, parameter_values=params).result().values[0]
```
b.
```python
ansatz_opt = transpile(ansatz, optimization_level=3)  # no backend
with Session(backend=backend):
    est = Estimator()
    value = est.run(ansatz_opt, ham, params).result()[0]
```
c.
```python
pm = generate_preset_pass_manager(optimization_level=3)  # no backend
ansatz_opt = pm.apply(ansatz)
with Session(backend=backend):
    est = Estimator()
    value = est(ansatz_opt, ham, params)
```
d.
```python
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
with Session(backend=backend):
    ansatz_opt = pm.transpile(ansatz)
    est = Estimator()
    value = est.run([ansatz_opt, ham, params]).values[0]
```


:::{dropdown}answer 
The answer is a. 

Use `generate_preset_pass_manager(backend=..., optimization_level=3)` and then `pm.run(circuit)` to obtain an optimized circuit before running it with a primitive inside a session. 
:::


14. Select the correct pattern to submit two jobs in a session and then retrieve their results **after** the session is closed.

a.
```python
session = Session(backend=backend)
estimator = Estimator(mode=session)
sampler = Sampler(mode=session)
job1 = estimator.run([est_pub])
job2 = sampler.run([sam_pub])
session.close()  # stop accepting new jobs
res1 = job1.result()
res2 = job2.result()
```
b.
```python
with Session(backend=backend) as session:
    estimator = Estimator()
    sampler = Sampler()
session.submit(estimator.run([est_pub]))
session.submit(sampler.run([sam_pub]))
res1 = job1.result(); res2 = job2.result()
```
c.
```python
session = Session(backend=backend)
estimator = Estimator()
sampler = Sampler()
job1 = estimator.run([est_pub])
job2 = sampler.run([sam_pub])
session.cancel()
res1 = job1.result(); res2 = job2.result()
```
d.
```python
with Session(backend=backend) as session:
    estimator = Estimator(mode=session)
    sampler = Sampler(mode=session)
    session.close()
    res1 = estimator.result(); res2 = sampler.result()
```


:::{dropdown}answer 
The answer is a. 

When not using a context manager, pass `mode=session` to primitives, submit jobs, then `session.close()` to stop accepting new jobs while allowing queued/running ones to complete; you can still call `.result()` afterward. This mirrors the documented example. 
:::


5. You are not using a context manager. Which snippet correctly **binds primitives to an existing session**?

a.
```python
session = Session(backend=backend)
estimator = Estimator(mode=session)
sampler = Sampler(mode=session)
```
b.
```python
session = Session(backend=backend)
estimator = Estimator(session=session)
sampler = Sampler(session=session)
```
c.
```python
session = Session(backend=backend)
estimator = Estimator(backend=session)
sampler = Sampler(backend=session)
```
d.
```python
estimator = Estimator()
sampler = Sampler()
with Session(backend=backend) as session:
    pass
```


:::{dropdown}answer 
The answer is a. 

The guide specifies `Estimator(mode=session)` (and similarly for `Sampler`) when not using a context manager. (It notes that versions earlier than 0.24.0 used `session=`.) 
:::


6. You plan an iterative VQE inside a session and want to pre-optimize the ansatz at **optimization level 3**. Which end-to-end pattern matches the guide?

a.
```python
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
ansatz_opt = pm.run(ansatz)
with Session(backend=backend) as session:
    est = Estimator()
    energy = est.run(ansatz_opt, hamiltonian, parameter_values=theta).result().values[0]
```
b.
```python
with Session(backend=backend) as session:
    pm = generate_preset_pass_manager(optimization_level=3)  # backend optional
    ansatz_opt = pm.run(ansatz)
    est = Estimator(mode=session)
    energy = est.run([ansatz_opt, hamiltonian, theta]).values[0]
```
c.
```python
with Session(backend=backend):
    est = Estimator()
energy = est.run([ansatz], hamiltonian, theta, optimization_level=3).result().values[0]
```
d.
```python
pm = generate_preset_pass_manager(backend=backend, level=3)
ansatz_opt = pm.transpile(ansatz)
with Session(backend=backend):
    est = Estimator()
    energy = est(ansatz_opt, hamiltonian, theta)
```


:::{dropdown}answer 
The answer is a. 

The session guide's VQE example shows generating a preset pass manager with `optimization_level=3`, running it on the circuit (`pm.run(...)`), and then using the optimized circuit with an `Estimator` inside a session. 
:::
